In [ ]:
import tweepy

# set up your Twitter API keys and access tokens
consumer_key = ""
consumer_secret = ""
access_token = ""
access_token_secret = ""


In [ ]:
import tweepy
import urllib.request
import cv2
import numpy as np

# Load the GenderNet model
gender_net = cv2.dnn.readNetFromCaffe("/content/drive/MyDrive/AgeGender/gender_deploy.prototxt", "/content/drive/MyDrive/AgeGender/gender_net.caffemodel")

# Authenticate with Twitter API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# Replace with the Twitter usernames to analyze
usernames = df['Username'].tolist()

# Create a new column in the DataFrame to hold the gender classification results
df['gender'] = np.nan

# Loop through the usernames and analyze their profile pictures
for username in usernames:
    # Retrieve the user's profile picture URL from their Twitter profile
    user = api.get_user(username)
    image_url = user.profile_image_url_https.replace("_normal", "")

    # Load the image from the URL and convert it to a numpy array
    with urllib.request.urlopen(image_url) as url:
        image_data = url.read()
    image_np = np.frombuffer(image_data, np.uint8)
    image = cv2.imdecode(image_np, cv2.IMREAD_COLOR)

    # Define the input size of the GenderNet model
    input_size = (227, 227)

    # Resize the image to the expected input size of the GenderNet model
    resized_image = cv2.resize(image, input_size)

    # Preprocess the image for the GenderNet model
    blob = cv2.dnn.blobFromImage(resized_image, scalefactor=1.0, size=input_size, mean=(78.4263377603, 87.7689143744, 114.895847746), swapRB=False)

    # Run the GenderNet model on the image
    gender_net.setInput(blob)
    gender_preds = gender_net.forward()

    # Classify the gender based on the output of the GenderNet model
    gender_prob = gender_preds[0][0]
    if gender_prob > 0.5:
        gender = "Male"
    else:
        gender = "Female"

    # Update the gender column in the DataFrame with the classification result
    df.loc[df['Username'] == username, 'gender'] = gender

# Print the updated DataFrame
print(df)
